# Example notebook with a FreePDK45 inverter

This needs to have the pdkmaster code installed in the environment where this notebook is executed.

In [ ]:
from matplotlib import pyplot, ticker

from pdkmaster.design import layout as lay, circuit as ckt
from pdkmaster.techs.freepdk45 import tech, layoutfab, cktfab

nmos = tech.primitives.nmos_vtg
pmos = tech.primitives.pmos_vtg

In [ ]:
# TODO: this should be defined by the technology
plotting = lay.Plotter({
    "pwell": {"fc": (1.0, 1.0, 0.0, 0.2), "ec": "orange", "zorder": 10},
    "nwell": {"fc": (0.0, 0.0, 0.0, 0.1), "ec": "grey", "zorder": 10},
    "active": {"fc": "lawngreen", "ec": "lawngreen", "zorder": 11},
    "poly": {"fc": "red", "ec": "red", "zorder": 12},
    "nimplant": {"fc": "purple", "ec": "purple", "alpha": 0.3, "zorder": 13},
    "pimplant": {"fc": "blueviolet", "ec": "blueviolet", "alpha": 0.3, "zorder": 13},
    "vthg": {"fc": (0.0, 0.0, 0.0, 0.0), "ec": "grey", "zorder": 13},
    "vthl": {"fc": (1, 1, 1, 0.3), "ec": "whitesmoke", "zorder": 13},
    "vthh": {"fc": (0.0, 0.0, 0.0, 0.2), "ec": "dimgrey", "zorder": 13},
    "contact": {"fc": "black", "ec": "black", "zorder": 14},
    "metal1": {"fc": (0.1, 0.1, 1, 0.4), "ec": "blue", "zorder": 15},
})

In [ ]:
from shapely import geometry as sh_geo
l = layoutfab(
    tech.primitives.contact, center=sh_geo.Point(0.2,0.2),
    **tech.primitives.contact.cast_params({
        "rows": 2, "columns": 2,
        "bottom": tech.primitives.active,
        "bottom_implant": tech.primitives.nimplant,
        "bottom_well": tech.primitives.pwell,
    }),
)
plotting.plot(l)
ax = pyplot.gca()
ax.axis("equal")

In [ ]:
l_n = nmos.computed.min_l
l_p = pmos.computed.min_l
w_n = nmos.computed.min_w
w_p = 2*w_n

#
# Create the circuit
#
inv = cktfab.new_circuit("inverter")
n1 = inv.new_instance("n1", nmos, l=l_n, w=w_n)
p1 = inv.new_instance("p1", pmos, l=l_p, w=w_p)
# n1 = inv.new_instance("n1", nmos, l=l_n, w=w_n, activeimplant_enclosure=0.020)
# p1 = inv.new_instance("p1", pmos, l=l_p, w=w_p, activeimplant_enclosure=0.020)
in_ = inv.new_net("IN", external=True)
out = inv.new_net("OUT", external=True)
vdd = inv.new_net("VDD", external=True)
gnd = inv.new_net("GND", external=True)
in_.childports += (n1.ports.gate, p1.ports.gate)
gnd.childports += (n1.ports.sourcedrain1, n1.ports.bulk)
vdd.childports += (p1.ports.sourcedrain1, p1.ports.bulk)
out.childports += (n1.ports.sourcedrain2, p1.ports.sourcedrain2)

#
# Create layout
#
contact = tech.primitives.contact
active = tech.primitives.active
poly = tech.primitives.poly
nimplant = tech.primitives.nimplant
pimplant = tech.primitives.pimplant
nwell = tech.primitives.nwell
pwell = tech.primitives.pwell
metal1 = tech.primitives.metal1

# TODO: Use packer to derive minimal placement of p transistor
impl_n_enc = nmos.min_gateimplant_enclosure[0] # This is not generic yet
impl_p_enc = pmos.min_gateimplant_enclosure[0] # This is not generic yet
y_p = 0.5*w_n + impl_n_enc.spec + impl_p_enc.spec + 0.5*w_p

x_v = max(
    0.5*l_n + nmos.computed.min_contactgate_space,
    0.5*l_p + pmos.computed.min_contactgate_space,
) + 0.5*contact.width

layouter = lay.CircuitLayouter(inv, layoutfab)

# gnd tap to pwell
layouter.add_wire(
    net=gnd, wire=contact, x=0, y=-0.2,
    bottom=active, bottom_implant=pimplant, bottom_well=pwell,
    top_width=0.4,
)
# vdd tap to nwell
layouter.add_wire(
    net=vdd, wire=contact, x=0, y=0.5,
    bottom=active, bottom_implant=nimplant, bottom_well=nwell,
    top_width=0.4,
)

# gnd contact to source of nmos
layouter.add_wire(
    net=gnd, well_net=gnd, wire=contact, x=-x_v, y=0.0,
    bottom=active, bottom_implant=nimplant, bottom_well=pwell,
    bottom_height=w_n,
)
# vdd contact to source of pmos
layouter.add_wire(
    net=vdd, well_net=vdd, wire=contact, x=-x_v, y=y_p,
    bottom=active, bottom_implant=pimplant, bottom_well=nwell,
    bottom_height=w_p,
)

# The two transistors
layouter.place(n1, x=0.0, y=0.0)
layouter.place(p1, x=0.0, y=y_p)

# out contact to drain of nmos
layouter.add_wire(
    net=out, well_net=gnd, wire=contact, x=x_v, y=0.0,
    bottom=active, bottom_implant=nimplant, bottom_well=pwell,
    bottom_height=w_n,
)
# out contact to drain of pmos
layouter.add_wire(
    net=out, well_net=vdd, wire=contact, x=x_v, y=y_p,
    bottom=active, bottom_implant=pimplant, bottom_well=nwell,
    bottom_height=w_p,
)

layouter.connect(masks=(
    nwell.mask, pwell.mask, poly.mask, metal1.mask,
))

pyplot.figure(figsize=(4.0, 7.0))

plotting.plot(layouter.layout)

ax = pyplot.gca()
ax.set_title("inverter")
ax.xaxis.set_major_locator(ticker.MultipleLocator(0.1))
ax.yaxis.set_major_locator(ticker.MultipleLocator(0.1))
ax.axis("equal")
ax.axis([-0.25, 0.25, -0.3, 0.6])
ax.grid("on")

pyplot.show()